## Importação de 

*   Bibliotecas
*   TEC
*   F10.7 e Sunspot

In [1]:
import requests,json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math, time
import pymysql.cursors

### Dados que são utilizados futuramente e conectando com o BD

In [2]:
lon = []
lat = []
ano_i = 2016
ano_f = 2019
step = -180
while step != 185:
    lon.append(step)
    step += 5
step = 87.5
while step != -90:
    lat.append(step)
    step -= 2.5
    
try:
    banco=pymysql.connect(
        host='localhost',
        user='root',
        password='',
        db='tecdata',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
except:
    print('error ao conectar com o banco de dados')

In [11]:
dias = 365
TEC = []
TEC_lat = []
for latitude in lat:
    for ano in range(ano_i, ano_f+1):
        db_name = str(ano)+'_'+str(latitude)
        with banco.cursor() as cursor:
            cursor.execute('SELECT * FROM `'+db_name+'` ORDER BY id ASC LIMIT 0,4745')
            dados = cursor.fetchall()
            TEC_lat = TEC_lat + dados

    TEC.append(TEC_lat)
    TEC_lat = []

In [ ]:
len(TEC[0])

In [4]:
RSN = []
factor = 0
for ano in range(ano_i,ano_f+1):
    with banco.cursor() as cursor:
        cursor.execute('SELECT id,`'+str(ano)+'`FROM rsn ORDER BY id ASC LIMIT 0,365')
        dados = cursor.fetchall()
        RSN = RSN + dados
    factor += 1
    
#Criando e colocando o RSN em uma lista
rsn = []
step = 0
for ano in range(ano_i,ano_f+1):
    for f in range(step, step+365):
        rsn.append(RSN[f][str(ano)])
    step=step+365

### Dados utilizados para prever o TEC

In [5]:
with banco.cursor() as cursor:
    cursor.execute('SELECT id,`'+str(2015)+'`FROM rsn ORDER BY id ASC LIMIT 0,365')
    RSN = cursor.fetchall()
sunspot_aprever = []
for f in range(0, 365):
    sunspot_aprever.append(RSN[f]["2015"])

## Desenvolvendo a DCT

In [6]:
import math, time
def dct(p):
    n = len(p)
    mat = []
    suma = 0
    cte=math.sqrt(2/n)
    cf = 0
    for i in range(n):
        if i==0: cf = 1/math.sqrt(2)
        else: cf = 1
        for t in range(n):
            theta = ((2*t+1)*i*math.pi)/(2*n)
            suma = suma + p[t]*math.cos(theta)
        suma = cf*suma*cte
        mat.append(np.round(suma, decimals=3))
        suma = 0
    return mat

def idct(g):
    n = len(g)
    cte = math.sqrt(2/n)
    imat = []
    suma = 0
    cj = 0
    for i in range(n):
        suma = 0
        for t in range(n):
            if t==0: cj=1/math.sqrt(2)
            else: cj=1
            theta = ((2*i+1)*t*math.pi)/(2*n)
            suma = suma + cj*g[t]*math.cos(theta)
        suma = suma*cte
        imat.append(np.round(suma, decimals=3))
    return imat

p = [12, 10, 8, 10, 12, 10, 8, 11, 67]
a = dct(p)
b = idct(a)
print(p,'\n\n', a, '\n\n',b)

[12, 10, 8, 10, 12, 10, 8, 11, 67] 

 [49.333, -25.942, 25.756, -22.454, 23.785, -16.257, 11.549, -8.459, 4.092] 

 [12.0, 10.0, 8.0, 10.0, 12.0, 10.0, 8.0, 11.001, 67.0]


## Fazendo a reconstrução

In [7]:
from sklearn.metrics import r2_score 

In [12]:
index = ['coef '+str(i) for i in range(1,14,1)] + ['Sunspot']
for latitude in range(1):
    for longitude in range(1):
        #Selecionando os dados para uma dada longitude
        dados = []
        dias = 365
        for i in range(13*dias*factor):
                dados.append(TEC[latitude][i][str(lon[longitude])])#Longitude -50.0
                
        TEC_f = pd.DataFrame(index=index)
        step = 0
        for dias in range(1,factor*dias+1):
            data = dados[step:step+13] + [rsn[dias-1]] 
            data = np.array(data).reshape(14,1)
            TEC_f[dias] = data
            step = step+13

        #Vamos aplicar aqui a DCT ao DataFrame TEC_f
        for i in range(1,dias+1):
            TEC_f[i][0:13] = dct(list(TEC_f[i][0:13]))  
        TEC_f = TEC_f.transpose()
        
        
        TEC_f_Previsto = pd.DataFrame(index=['coef '+str(i) for i in range(1,14,1)])
        TEC_f_Previsto = TEC_f_Previsto.transpose()
        for coeficiente in range(1,14):
            curve = np.poly1d(np.polyfit(TEC_f['Sunspot'], TEC_f['coef '+str(coeficiente)],6))
            previsao = curve(sunspot_aprever)
            TEC_f_Previsto['coef '+str(coeficiente)] = previsao
            curve = []
            previsao = []

In [9]:
TEC_f

,coef 1,coef 2,coef 3,coef 4,coef 5,coef 6,coef 7,coef 8,coef 9,coef 10,coef 11,coef 12,coef 13,Sunspot
1,118.151,-10.995,2.631,13.549,2.591,2.145,7.230,-16.098,-14.752,-6.376,-2.636,-0.326,-2.971,114.0
2,148.660,-14.822,1.480,-5.395,9.415,16.045,-3.949,-1.212,-5.238,2.864,11.291,-12.107,4.056,114.4
3,177.781,-49.493,32.652,9.641,-9.488,25.672,-13.966,4.975,-9.582,2.919,-5.343,-4.022,11.232,114.8
4,133.405,12.334,1.522,10.958,6.044,14.617,5.815,-3.154,9.179,0.959,3.760,1.129,0.972,115.0
5,196.364,-28.229,-13.934,8.839,-15.444,9.243,-6.848,-21.502,2.951,-6.607,9.392,-9.980,-5.172,118.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,74.052,5.583,-11.435,-4.582,-1.350,-5.515,0.439,2.138,-1.987,0.868,0.235,0.775,1.402,86.6
1457,68.505,2.304,-9.594,-7.643,-1.354,-0.699,-1.982,3.135,-0.113,-0.124,1.216,-1.716,1.305,86.8
1458,65.732,7.263,-8.161,-4.599,-0.845,-2.955,-2.068,-0.476,-1.534,0.587,0.195,-1.816,1.957,86.4
1459,85.424,-5.490,-14.328,-7.524,-1.666,-1.297,0.930,-0.382,-2.957,2.498,-2.017,-0.432,0.190,85.1


In [13]:
teste_lat_tec = []
teste_tec = []
for latitude in lat[0:1]:
    for ano in range(2015,2016):
        db_name = str(ano)+'_'+str(latitude)
        with banco.cursor() as cursor:
            cursor.execute('SELECT * FROM `'+db_name+'` ORDER BY id ASC LIMIT 0,4745')
            dados = cursor.fetchall()
            teste_lat_tec = teste_lat_tec + dados
    teste_tec.append(teste_lat_tec)
    teste_lat_tec = []
    
dados = []
dias = 365
for i in range(13*dias):
    dados.append(teste_tec[0][i][str(lon[0])])#Longitude -50.0

index = ['dado '+str(i) for i in range(1,14,1)] 
TEC_t = pd.DataFrame(index=index)
step = 0
for dias in range(1,dias+1):
    data = dados[step:step+13] 
    data = np.array(data).reshape(13,1)
    TEC_t[dias] = data
    step = step+13

In [14]:
TEC_t

,1,2,3,4,5,6,7,8,9,10,...,356,357,358,359,360,361,362,363,364,365
dado 1,72,72,108,34,80,68,70,81,95,109,...,54,36,40,55,55,48,38,63,55,73
dado 2,45,62,130,41,79,87,68,84,72,83,...,47,46,54,55,43,39,44,51,37,66
dado 3,38,66,94,38,56,59,81,64,67,71,...,41,42,48,49,45,42,31,43,23,53
dado 4,51,61,65,59,42,57,71,70,52,61,...,50,39,50,48,53,47,30,36,18,48
dado 5,53,71,65,62,56,62,93,77,50,60,...,45,54,72,67,63,58,49,30,28,70
dado 6,47,67,68,73,59,100,169,77,67,68,...,42,59,58,57,44,66,58,35,41,72
dado 7,49,101,60,92,57,70,161,88,67,91,...,54,48,55,47,52,54,43,28,44,76
dado 8,41,100,64,124,55,73,95,89,82,100,...,54,58,52,35,40,39,37,33,45,84
dado 9,72,75,57,144,53,93,64,105,104,111,...,51,63,52,52,47,40,38,66,58,54
dado 10,55,55,52,164,52,76,63,81,128,137,...,48,52,47,40,36,40,33,53,59,42


In [26]:
TEC_f_Previsto = TEC_f_Previsto.transpose()
idct(np.array(TEC_f_Previsto[0]).reshape(-1,1))

[array([106.467]),
 array([133.989]),
 array([165.557]),
 array([120.462]),
 array([74.697]),
 array([98.133]),
 array([168.63]),
 array([187.353]),
 array([174.61]),
 array([156.595]),
 array([150.633]),
 array([153.581]),
 array([93.41])]